**Makeat Ai모델 학습과정**

In [1]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.version

'3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]'

In [2]:
%cd yolov5
%pip install -qr requirements.txt

c:\Makeat\yolov5
Note: you may need to restart the kernel to use updated packages.


In [4]:
import json
import os
import shutil
import cv2
import glob
import re
import yaml
import fnmatch
import wandb
import random
import torch
import pandas as pd
import numpy as np

In [5]:
!wandb login

wandb: Currently logged in as: ben8169. Use `wandb login --relogin` to force relogin


In [6]:
wandb.init(project="Makeat", reinit=True)
wandb.run.name = 'run1'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ben8169. Use `wandb login --relogin` to force relogin


1. 다운로드 받은 training set , validation set 임시 파일에 이동 

In [2]:
x = glob.glob("C:\\Makeat\\tmp\\*")
x

['C:\\Makeat\\tmp\\[라벨]음식001_Tra_json',
 'C:\\Makeat\\tmp\\[라벨]음식001_Val_json',
 'C:\\Makeat\\tmp\\[원천]음식001_Tra',
 'C:\\Makeat\\tmp\\[원천]음식001_Val']

In [3]:
tmp_dir = "C:\\Makeat\\tmp\\"

dir_list = os.listdir(tmp_dir)

i = [f for f in dir_list if fnmatch.fnmatch(f, '[원천**_Tra')]
train_img_dir = os.path.join(tmp_dir,i[0])
print(train_img_dir)

i = [f for f in dir_list if fnmatch.fnmatch(f, '[원천**_Val')]
valid_img_dir = os.path.join(tmp_dir,i[0])
print(valid_img_dir)

i = [f for f in dir_list if fnmatch.fnmatch(f, '[라벨**_Tra_json')]
train_lbl_dir = os.path.join(tmp_dir,i[0])
print(train_lbl_dir)

i = [f for f in dir_list if fnmatch.fnmatch(f, '[라벨**_Val_json')]
valid_lbl_dir = os.path.join(tmp_dir,i[0])
print(valid_lbl_dir)

C:\Makeat\tmp\[원천]음식001_Tra
C:\Makeat\tmp\[원천]음식001_Val
C:\Makeat\tmp\[라벨]음식001_Tra_json
C:\Makeat\tmp\[라벨]음식001_Val_json


In [5]:
with open("C:\\Makeat\\data.yaml", 'r', encoding='utf-8') as f:
    data = yaml.safe_load(f)
    
nc = data['nc']

In [5]:
foodlist = os.listdir(train_img_dir)
foodlist

['가리비',
 '갈비탕',
 '골드키위',
 '과일채소샐러드',
 '그린올리브',
 '김구이',
 '달걀말이',
 '닭갈비',
 '닭살채소두반장볶음',
 '닭파프리카볶음',
 '도리야끼',
 '돼지감자',
 '돼지고기볶음',
 '마르게리따피자',
 '마시멜로우',
 '마카롱',
 '메쉬드포테이토',
 '멜론',
 '모듬초밥',
 '모짜렐라치즈',
 '무우',
 '미소장국',
 '미역나물',
 '바게트빵',
 '바나나우유',
 '보드카',
 '보이차',
 '붉은 양배추 절임',
 '브라질너트',
 '상추샐러드',
 '생맥주',
 '송편',
 '순대국밥',
 '순살찜닭',
 '슈크림',
 '스크램블드에그',
 '스테이크',
 '스튜']

이미지와 라벨 이름 일치하지 않는 것 거르기 + 폴더이름 틀린것까지 검증

In [11]:
for food in foodlist:
    json_dir = os.path.join(train_lbl_dir,food+" json")
    img_dir = os.path.join(train_img_dir,food)
    json_dir2= os.path.join(valid_lbl_dir,food+" json")
    img_dir2 = os.path.join(valid_img_dir,food)

    correct_train_list = [f for f in os.listdir(json_dir) if os.path.isfile(os.path.join(img_dir, os.path.splitext(os.path.basename(f))[0] + '.jpg'))]
    correct_valid_list = [f for f in os.listdir(json_dir2) if os.path.isfile(os.path.join(img_dir2, os.path.splitext(os.path.basename(f))[0] + '.jpg'))]


training data 부터 처리 

In [13]:
correct_train_list = []
correct_valid_list = []
print(nc)


0


3. json to txt, 이름 일치하도록 image/label 구성(validation set의 1/2은 test set으로 설정)

In [14]:
for food in foodlist:
    #training data
    json_dir = os.path.join(train_lbl_dir,food+" json")
    img_dir = os.path.join(train_img_dir,food)
    correct_train_list = [f for f in os.listdir(json_dir)]
    correct_foodname_list = [f.replace(".json","") for f in correct_train_list]

    for file in correct_foodname_list:
        #이미지 이동
        shutil.move(os.path.join(img_dir, file+'.jpg'),"C:\\Makeat\\dataset\\train\\images")

        #txt 생성 후 저장
        with open(os.path.join(json_dir, file+'.json')) as f:
            d = json.load(f)

            x, y = map(float, d[0]['Point(x,y)'].split(','))
            w = float(d[0]['W'])
            h = float(d[0]['H'])
            
            txt_file_name = file + '.txt'
            txt_file_path = 'C:\\Makeat\\dataset\\train\\labels\\' + txt_file_name
            with open(txt_file_path, 'w') as tfile:
              tfile.write(f"{nc} {x} {y} {w} {h}")
    
    # #valid data 처리 + test하고 나누기
    json_dir2= os.path.join(valid_lbl_dir,food+" json")
    img_dir2 = os.path.join(valid_img_dir,food)
    target_dir1 = "C:\\Makeat\\dataset\\valid\\"
    target_dir2 = "C:\\Makeat\\dataset\\test\\"

    correct_valid_list = [f for f in os.listdir(json_dir2) if os.path.isfile(os.path.join(img_dir2, os.path.splitext(os.path.basename(f))[0] + '.jpg'))]
    correct_foodname_list2 = [f.replace(".json","") for f in correct_valid_list]

    
    i = -1
    for file2 in correct_foodname_list2:
        #test와 valid 번갈아가며 파일 이동
        if i > 0:
            target = target_dir1
        else:
            target = target_dir2 #C:\\Makeat\\dataset\\test\\images

        #이미지 이동
        shutil.move(os.path.join(img_dir2, file2+'.jpg'),target+"images")

        #txt 생성 후 저장
        with open(os.path.join(json_dir2, file2+'.json')) as f:
            d2 = json.load(f)

            x, y = map(float, d2[0]['Point(x,y)'].split(','))
            w = float(d2[0]['W'])
            h = float(d2[0]['H'])
            
            txt_file_name = file2 + '.txt'
            txt_file_path = target + '\\labels\\' + txt_file_name
            with open(txt_file_path, 'w') as textfile:
              textfile.write(f"{nc} {x} {y} {w} {h}")

        i *= -1


    #yaml 추가용 딕셔너리 생성 후 라벨링 기록
    data['names'][nc] = food
    nc += 1

 

4. 이동한 파일 폴더 이름을 통해 음식 이름 numerize >> data.yaml 파일 수정 

In [7]:
print(data['names'])
print(len(data['names']))
print(nc)


with open("C:\Makeat\data.yaml", 'w', encoding='utf-8') as f:
    data['nc'] = nc
    yaml.safe_dump(data, f, allow_unicode=True)
    display(data)

{0: '가리비', 1: '갈비탕', 2: '골드키위', 3: '과일채소샐러드', 4: '그린올리브', 5: '김구이', 6: '달걀말이', 7: '닭갈비', 8: '닭살채소두반장볶음', 9: '닭파프리카볶음', 10: '도리야끼', 11: '돼지감자', 12: '돼지고기볶음', 13: '마르게리따피자', 14: '마시멜로우', 15: '마카롱', 16: '메쉬드포테이토', 17: '멜론', 18: '모듬초밥', 19: '모짜렐라치즈', 20: '무우', 21: '미소장국', 22: '미역나물', 23: '바게트빵', 24: '바나나우유', 25: '보드카', 26: '보이차', 27: '붉은 양배추 절임', 28: '브라질너트', 29: '상추샐러드', 30: '생맥주', 31: '송편', 32: '순대국밥', 33: '순살찜닭', 34: '슈크림', 35: '스크램블드에그', 36: '스테이크', 37: '스튜', 38: '달걀', 39: '달걀후라이', 40: '닭가슴살', 41: '당근', 42: '메밀', 43: '바나나', 44: '밥', 45: '빵', 46: '사과', 47: '상추', 48: '오트밀', 49: '치즈', 50: '토스트', 51: '통밀빵', 52: '김', 53: '냉면', 54: '닭가슴살샐러드', 55: '돼지고기구이', 56: '두부', 57: '삶은달걀', 58: '삼겹살', 59: '새우', 60: '토마토샐러드'}
61
61


{'names': {0: '가리비',
  1: '갈비탕',
  2: '골드키위',
  3: '과일채소샐러드',
  4: '그린올리브',
  5: '김구이',
  6: '달걀말이',
  7: '닭갈비',
  8: '닭살채소두반장볶음',
  9: '닭파프리카볶음',
  10: '도리야끼',
  11: '돼지감자',
  12: '돼지고기볶음',
  13: '마르게리따피자',
  14: '마시멜로우',
  15: '마카롱',
  16: '메쉬드포테이토',
  17: '멜론',
  18: '모듬초밥',
  19: '모짜렐라치즈',
  20: '무우',
  21: '미소장국',
  22: '미역나물',
  23: '바게트빵',
  24: '바나나우유',
  25: '보드카',
  26: '보이차',
  27: '붉은 양배추 절임',
  28: '브라질너트',
  29: '상추샐러드',
  30: '생맥주',
  31: '송편',
  32: '순대국밥',
  33: '순살찜닭',
  34: '슈크림',
  35: '스크램블드에그',
  36: '스테이크',
  37: '스튜',
  38: '달걀',
  39: '달걀후라이',
  40: '닭가슴살',
  41: '당근',
  42: '메밀',
  43: '바나나',
  44: '밥',
  45: '빵',
  46: '사과',
  47: '상추',
  48: '오트밀',
  49: '치즈',
  50: '토스트',
  51: '통밀빵',
  52: '김',
  53: '냉면',
  54: '닭가슴살샐러드',
  55: '돼지고기구이',
  56: '두부',
  57: '삶은달걀',
  58: '삼겹살',
  59: '새우',
  60: '토마토샐러드'},
 'nc': 61,
 'test': 'C:\\Makeat\\dataset\\test\\images',
 'train': 'C:\\Makeat\\dataset\\train\\images',
 'val': 'C:\\Makeat\\dataset\\valid\\images'}

4. 학습 및 확인 

In [21]:
#학습전 메모리 비우기>> 메모리 초과 방지
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [15]:
torch.cuda.is_available() 

True

In [18]:
torch.__version__

'1.13.1'

In [23]:
#최종 best모델 반복학습
!python train.py --img 640 --batch 16 --epochs 100 --data "C:\Makeat\dataset\data.yaml" --weights "C:\Makeat\yolov5\runs\train\exp31\weights\best.pt" --device 0

^C


5. 통합식품영양성분 DB 이용하여 영양소정보 수집

In [4]:
import chardet

with open("C:\Makeat\통합 식품영양성분DB.csv", 'rb') as f:
    result = chardet.detect(f.read())

df = pd.read_csv("C:\Makeat\통합 식품영양성분DB.csv",encoding='UTF-8-SIG')
df

C:\Users\ben81\AppData\Local\Temp\ipykernel_20796\3953266633.py:6: DtypeWarning: Columns (13,15,19,20,21,22,45,133,136,217) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:\Makeat\통합 식품영양성분DB.csv",encoding='UTF-8-SIG')


,NO,SAMPLE_ID,식품코드,DB군,상용제품,식품명,연도,지역 / 제조사,채취시기,식품대분류,...,냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),폐기율(%),가식부(%),산가용성물질(%),카페인(㎎),성분표출처,발행기관
0,1,D000006-94-AVG,D000006,음식,품목대표,꿩불고기,2019,충주,평균,구이류,...,-,-,-,5.8,-,-,-,-,식약처('16) 제4권,식품의약품안전처
1,2,D000007-ZZ-AVG,D000007,음식,품목대표,닭갈비,2019,전국(대표),평균,구이류,...,-,-,-,7,-,-,-,-,식약처('16) 제4권,식품의약품안전처
2,3,D000008-66-AVG,D000008,음식,품목대표,닭갈비,2019,춘천,평균,구이류,...,-,-,-,4.5,-,-,-,-,식약처('16) 제4권,식품의약품안전처
3,4,D000009-ZZ-AVG,D000009,음식,품목대표,닭꼬치,2019,전국(대표),평균,구이류,...,-,-,-,1.218,-,-,-,-,식약처('12) 제1권,식품의약품안전처
4,5,D000010-ZZ-AVG,D000010,음식,품목대표,더덕구이,2019,전국(대표),평균,구이류,...,-,-,-,2.5,-,-,-,-,식약처('17) 제5권,식품의약품안전처
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52935,52936,P057835-ZZ-AVG,P057835,가공식품,상용제품,라임모히또 젤리,2020,㈜청우식품,평균,캔디류,...,-,-,-,-,-,-,-,-,식약처('20),식품의약품안전처
52936,52937,P057836-ZZ-AVG,P057836,가공식품,상용제품,꼬마쮸 복숭아맛,2020,(유)대현물산,평균,캔디류,...,-,-,-,-,-,-,-,-,식약처('20),식품의약품안전처
52937,52938,P057837-ZZ-AVG,P057837,가공식품,상용제품,올리렉스 로젠지,2020,고궁,평균,캔디류,...,-,-,-,-,-,-,-,-,식약처('20),식품의약품안전처
52938,52939,P057838-ZZ-AVG,P057838,가공식품,상용제품,블루베리캔디,2020,㈜청우식품,평균,캔디류,...,-,-,-,-,-,-,-,-,식약처('20),식품의약품안전처


In [5]:
df = df[['식품명','1회제공량','에너지(㎉)','탄수화물(g)','단백질(g)','지방(g)','나트륨(㎎)']]
df

,식품명,1회제공량,에너지(㎉),탄수화물(g),단백질(g),지방(g),나트륨(㎎)
0,꿩불고기,500.0,368.8,39.7,33.5,8.5,1264.31
1,닭갈비,400.0,595.61,44.9,45.9,25.8,1535.83
2,닭갈비,300.0,558.47,23.1,45.5,31.6,1016.94
3,닭꼬치,70.0,176.723,13.348,11.562,8.565,286.911
4,더덕구이,100.0,184.0,31.1,3.1,5.2,743.37
...,...,...,...,...,...,...,...
52935,라임모히또 젤리,70.0,264,61,5,0,0
52936,꼬마쮸 복숭아맛,40.0,175,36,0,3.6,0
52937,올리렉스 로젠지,120.0,300,120,0,0,0
52938,블루베리캔디,100.0,390,97,0,0,0


존재하는 음식들에 한해 영양성분 수집

In [17]:
existing_food = []
non_existing_food = []
x = list(data['names'].values())
for i in x:
    if i in df['식품명'].values:
        existing_food.append(i)
    else:
        non_existing_food.append(i)

print(existing_food)
print(non_existing_food)


['갈비탕', '골드키위', '달걀말이', '닭갈비', '돼지고기볶음', '마르게리따피자', '마카롱', '모짜렐라치즈', '바나나우유', '보이차', '생맥주', '순대국밥', '슈크림', '스크램블드에그', '당근', '바나나', '사과', '통밀빵']
['가리비', '과일채소샐러드', '그린올리브', '김구이', '닭살채소두반장볶음', '닭파프리카볶음', '도리야끼', '돼지감자', '마시멜로우', '메쉬드포테이토', '멜론', '모듬초밥', '무우', '미소장국', '미역나물', '바게트빵', '보드카', '붉은 양배추 절임', '브라질너트', '상추샐러드', '송편', '순살찜닭', '스테이크', '스튜', '달걀', '달걀후라이', '닭가슴살', '메밀', '밥', '빵', '상추', '오트밀', '치즈', '토스트']


In [10]:
makeat_data = {'식품명':[],
'1회제공량':[],
'에너지(㎉)':[],
'탄수화물(g)':[],
'단백질(g)':[],
'지방(g)':[],
'나트륨(㎎)':[]
}


In [ ]:
x = list(data['names'].values())
makeat_data['식품명'] = x
makeat_data

for key in makeat_data.keys():
    if key != '식품명':
        makeat_data[key] = [0] * len(makeat_data['식품명'])

makeat_csv = pd.DataFrame(makeat_data)

In [27]:
for foodname in existing_food:
    makeat_index = makeat_csv.loc[makeat_csv['식품명'] == foodname].index[0]
    df_index = df.loc[df['식품명'] == foodname].index[0]
    print(foodname, makeat_index, df_index)
    makeat_csv.iloc[makeat_index] = df.iloc[df_index]
makeat_csv

갈비탕 1 6838
골드키위 2 41926
달걀말이 6 7187
닭갈비 7 1
돼지고기볶음 12 150
마르게리따피자 13 45727
마카롱 15 3223
모짜렐라치즈 19 23165
바나나우유 24 13458
보이차 26 10311
생맥주 30 5850
순대국밥 32 6997
슈크림 34 12391
스크램블드에그 35 7196
당근 41 43931
바나나 43 1052
사과 46 21115
통밀빵 51 6220


,식품명,1회제공량,에너지(㎉),탄수화물(g),단백질(g),지방(g),나트륨(㎎)
0,꿩불고기,500,368.8,39.7,33.5,8.5,1264.31
1,갈비탕,670,363,2.7,57,13.8,1330.59
2,골드키위,500,500,1,0,53,220
3,과일채소샐러드,0,0.0,0,0,0,0
4,그린올리브,0,0.0,0,0,0,0
5,김구이,0,0.0,0,0,0,0
6,달걀말이,100,193,1.4,17.2,13.2,449.07
7,닭갈비,400,595.61,44.9,45.9,25.8,1535.83
8,닭살채소두반장볶음,0,0.0,0,0,0,0
9,닭파프리카볶음,0,0.0,0,0,0,0


In [ ]:
#추후 식약청 DB 이용하여 csv에 직접 영양성분 추가
makeat_csv.to_csv('C:\\Makeat\\makeat.csv', index=False, encoding='UTF-8-SIG')

detection 테스트

In [14]:
model = torch.hub.load('ultralytics/yolov5', 'custom', 'C:/Makeat/yolov5/runs/train/exp32/weights/best.pt')  # custom trained model

image_path = "C:\\Users\\ben81\\OneDrive\\바탕 화면\\Kitty_Gun.jpg"
save_dir = 'C:\\Makeatresult\\'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
img = Image.open(image_path)

results = model(img)

results.save(save_dir=save_dir, exist_ok=True)

results.pandas().xyxy[0]
#detection 결과를 list로 return
list(results.pandas().xyxy[0]['name'])

Using cache found in C:\Users\ben81/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-2-21 Python-3.9.13 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 157 layers, 7112611 parameters, 0 gradients, 16.1 GFLOPs
Adding AutoShape... 
Saved 1 image to C:\Makeatresult


[]